##### Name: Usama Khalid
##### Email: i191236@nu.edu.pk

### 1. Importing packages and initializing variables

In [38]:
from sklearn.svm import SVC
import librosa
import glob
import os
import numpy as np
from sklearn.metrics import classification_report
data_dir = 'SERDataset/'
emotions = ['Angry','Happy','Neutral','Sad']

if os.path.isdir(data_dir+'Test/Neural/'):
    os.rename(data_dir+'Test/Neural/',data_dir+'Test/Neutral')

### 2. Define feature extraction function

In [39]:
def extract_features(file_path):
    signal , sampling_rate = librosa.load(file_path, sr=16000)
    chroma_stft = librosa.feature.chroma_stft(y=signal, sr=sampling_rate)
    rms = librosa.feature.rms(y=signal)
    spec_cent = librosa.feature.spectral_centroid(y=signal, sr=sampling_rate)
    spec_bw = librosa.feature.spectral_bandwidth(y=signal, sr=sampling_rate)
    rolloff = librosa.feature.spectral_rolloff(y=signal, sr=sampling_rate)
    zcr = librosa.feature.zero_crossing_rate(signal)
    mfcc = librosa.feature.mfcc(y=signal, sr=sampling_rate)
    features = [np.mean(chroma_stft),np.mean(rms),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr)]
    for e in mfcc:
        features.append(np.mean(e))

    return features

extract_features('SERDataset/Train/Happy/SF1_F4_H04.wav')

[0.20420063,
 0.12748179,
 1292.1764097284686,
 1383.7299256956453,
 2690.907579787234,
 0.06560110538563829,
 -177.66687,
 110.12055,
 -9.45184,
 47.768612,
 -19.158928,
 -0.7120025,
 -17.370464,
 8.52397,
 -9.810286,
 10.364548,
 3.9621286,
 -13.749555,
 3.3712375,
 -10.224567,
 -12.50654,
 1.9842318,
 -9.655012,
 -0.08594129,
 -0.76819074,
 11.497403]

### 3. Extract Features and labels


In [40]:
def get_data(path):
    return [(extract_features(file_path),emotions.index(emotion))
            for emotion in emotions
            for file_path in glob.glob(data_dir + path + emotion + '/*')]



### 4. Train Classifier

In [41]:
X_train, y_train = map(list, zip(*get_data('Train/')))
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)

### 5. Print classification performance statistics

In [42]:
X_test, y_test = map(list, zip(*get_data('Test/')))
y_pred = svm_model_linear.predict(X_test)
print(classification_report(y_test, y_pred, target_names=emotions))




              precision    recall  f1-score   support

       Angry       0.94      1.00      0.97        15
       Happy       0.88      1.00      0.94        15
     Neutral       1.00      0.93      0.97        15
         Sad       1.00      0.87      0.93        15

    accuracy                           0.95        60
   macro avg       0.95      0.95      0.95        60
weighted avg       0.95      0.95      0.95        60

